In [10]:
import pandas as pd
import numpy as np
df = pd.read_csv('data.csv', sep='|')

In [11]:
df.columns
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138047 entries, 0 to 138046
Data columns (total 57 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Name                         138047 non-null  object 
 1   md5                          138047 non-null  object 
 2   Machine                      138047 non-null  int64  
 3   SizeOfOptionalHeader         138047 non-null  int64  
 4   Characteristics              138047 non-null  int64  
 5   MajorLinkerVersion           138047 non-null  int64  
 6   MinorLinkerVersion           138047 non-null  int64  
 7   SizeOfCode                   138047 non-null  int64  
 8   SizeOfInitializedData        138047 non-null  int64  
 9   SizeOfUninitializedData      138047 non-null  int64  
 10  AddressOfEntryPoint          138047 non-null  int64  
 11  BaseOfCode                   138047 non-null  int64  
 12  BaseOfData                   138047 non-null  int64  
 13 

In [12]:
df["legitimate"].value_counts()

,count
legitimate,
0,96724
1,41323


In [16]:
from sklearn.model_selection import train_test_split
import sklearn.ensemble as ek
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier
import joblib
import pickle
import os


In [17]:
X = df.drop(['Name','md5','legitimate'],axis=1).values
y = df['legitimate'].values
len(y)

138047

In [18]:
extratrees = ek.ExtraTreesClassifier().fit(X,y)

In [19]:
model = SelectFromModel(extratrees, prefit=True)
X_new = model.transform(X)
nbfeatures = X_new.shape[1]
nbfeatures


13

In [20]:
X_train, X_test,y_train, y_test = train_test_split(X_new, y ,test_size=0.29, stratify = y)

In [21]:
features = []
index = np.argsort(extratrees.feature_importances_)[::-1][:nbfeatures]
for f in range(nbfeatures):
  print("%d, feature %s (%f)" % ((f+1), df.columns[2+index[f]], extratrees.feature_importances_[index[f]]))
  features.append(df.columns[2+index[f]])

1, feature DllCharacteristics (0.169490)
2, feature Machine (0.124266)
3, feature Characteristics (0.116258)
4, feature VersionInformationSize (0.061396)
5, feature Subsystem (0.059565)
6, feature ImageBase (0.052604)
7, feature MajorSubsystemVersion (0.049316)
8, feature SectionsMaxEntropy (0.045296)
9, feature ResourcesMaxEntropy (0.040200)
10, feature SizeOfOptionalHeader (0.036104)
11, feature ResourcesMinEntropy (0.029039)
12, feature SizeOfStackReserve (0.023799)
13, feature MajorOperatingSystemVersion (0.021980)


In [22]:
model = {
    "DecisionTree" : DecisionTreeClassifier(max_depth=10),
    "RandomForest" : ek.RandomForestClassifier(n_estimators=50),
}

In [23]:
results = {}
for algo in model:
  clf = model[algo]
  clf.fit(X_train, y_train)
  score = clf.score(X_test,y_test)
  print("%s:  %s " % (algo, score))
  results[algo] = score


DecisionTree:  0.9907578558225508 
RandomForest:  0.9938802018284458 


In [24]:
best = max(results, key=results.get)
print(best)
joblib.dump(model[best], 'Classifier.pkl')

RandomForest


['Classifier.pkl']

In [25]:
open('features.pkl', 'wb').write(pickle.dumps(features))
clf = joblib.load('Classifier.pkl')
features = pickle.loads(open(os.path.join('features.pkl'), 'rb').read())